TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [157]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [158]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [159]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'
tag_name = 'all'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
    data_path = f'{root_path}'
else:
    root_path = '../'
    data_path = f'{root_path}data/'
    
model_path = f'{root_path}models/'
logs_path = f'{root_path}logs/'
    
sequence_length = 40

In [160]:
with open(f'{data_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [161]:
corpus[0:500]

"Good morning. How are you?  Good. It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.  There have been three themes running through the conference, which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here; just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen in te"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [162]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—', '%','[', ']' ]

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '$', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'è', 'é', '–', '’', '“', '”', '…', '♫']


In [163]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [164]:
# If running models with punctuation removed, leave commented out
#chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 75


In [165]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '$': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'W': 37, 'X': 38, 'Y': 39, 'Z': 40, 'a': 41, 'b': 42, 'c': 43, 'd': 44, 'e': 45, 'f': 46, 'g': 47, 'h': 48, 'i': 49, 'j': 50, 'k': 51, 'l': 52, 'm': 53, 'n': 54, 'o': 55, 'p': 56, 'q': 57, 'r': 58, 's': 59, 't': 60, 'u': 61, 'v': 62, 'w': 63, 'x': 64, 'y': 65, 'z': 66, 'è': 67, 'é': 68, '–': 69, '’': 70, '“': 71, '”': 72, '…': 73, '♫': 74}


In [166]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '$', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'Q', 32: 'R', 33: 'S', 34: 'T', 35: 'U', 36: 'V', 37: 'W', 38: 'X', 39: 'Y', 40: 'Z', 41: 'a', 42: 'b', 43: 'c', 44: 'd', 45: 'e', 46: 'f', 47: 'g', 48: 'h', 49: 'i', 50: 'j', 51: 'k', 52: 'l', 53: 'm', 54: 'n', 55: 'o', 56: 'p', 57: 'q', 58: 'r', 59: 's', 60: 't', 61: 'u', 62: 'v', 63: 'w', 64: 'x', 65: 'y', 66: 'z', 67: 'è', 68: 'é', 69: '–', 70: '’', 71: '“', 72: '”', 73: '…', 74: '♫'}


In [167]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 267,106 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [168]:
corpus[0:50]

'Good morning. How are you?  Good. Its been great h'

In [169]:
corpus[1:41]

'ood morning. How are you?  Good. Its bee'

In [170]:
corpus[41]

'n'

In [171]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [172]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[21, 55, 55, 44, 0, 53, 55, 58, 54, 49, 54, 47, 3, 0, 22, 55, 63, 0, 41, 58, 45, 0, 65, 55, 61, 14, 0, 0, 21, 55, 55, 44, 3, 0, 23, 60, 59, 0, 42, 45]
[55, 55, 44, 0, 53, 55, 58, 54, 49, 54, 47, 3, 0, 22, 55, 63, 0, 41, 58, 45, 0, 65, 55, 61, 14, 0, 0, 21, 55, 55, 44, 3, 0, 23, 60, 59, 0, 42, 45, 45]


In [173]:
print ( y_numeric_list[0:2])

[45, 54]


In [174]:
indices_char[51]

'k'

In [175]:
corpus[0:43]

'Good morning. How are you?  Good. Its been '

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [176]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 267,066 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [177]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [178]:
X.shape

(267066, 40, 1)

In [179]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [180]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [181]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [182]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

In [183]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])


#Models are saved in this format:
# model.save(f'{model_path}/{tag_name}_{input_nodes_count}_{layer_2_nodes_count}')
print(f'Loading model:{model_path}{tag_name}_40_256')
model = load_model(f'{model_path}{tag_name}_40_256')

model.summary()

Loading model:../models/all_40_256
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 40, 800)           2566400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 800)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               1082368   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 75)                19275     
Total params: 3,668,043
Trainable params: 3,668,043
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [184]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 221999 sequence.
[0, 41, 54, 44, 0, 63, 55, 58, 51, 0, 55, 54, 0, 41, 54, 65, 60, 48, 49, 54, 47, 0, 65, 55, 61, 0, 63, 41, 54, 60, 0, 41, 59, 0, 52, 55, 54, 47, 0, 41]


Convert this back to text so we can see what it looks like.

In [185]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

' and work on anything you want as long a'

In [186]:
# We can also use the seed_start value to see a larger picture of the original text
for j in range(seed_start, seed_start + 15):
    seed_numeric = X_numeric_list[j]
    print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

 and work on anything you want as long a
and work on anything you want as long as
nd work on anything you want as long as 
d work on anything you want as long as i
 work on anything you want as long as it
work on anything you want as long as its
ork on anything you want as long as its 
rk on anything you want as long as its n
k on anything you want as long as its no
 on anything you want as long as its not
on anything you want as long as its not 
n anything you want as long as its not p
 anything you want as long as its not pa
anything you want as long as its not par
nything you want as long as its not part


To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [187]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 75)
Prediction:
[[9.16381032e-05 3.37330112e-25 1.33921819e-26 1.21753677e-10
  3.05293238e-26 4.21590061e-28 3.34571277e-23 2.94933621e-25
  5.82045057e-26 1.09997857e-24 1.08403979e-26 3.90648136e-22
  8.20134532e-22 1.45579234e-21 7.14561362e-17 1.04308561e-15
  3.09624482e-19 3.91852884e-28 5.59942607e-15 9.76604212e-16
  2.40938891e-20 4.10525273e-35 6.05848050e-18 2.16088367e-12
  1.90128333e-37 1.95608585e-13 7.54559002e-20 4.34552466e-16
  3.89135491e-15 7.70538196e-17 1.32061305e-14 2.87813326e-20
  1.55609756e-22 6.86104212e-12 9.51424498e-20 4.52326295e-19
  3.12671480e-18 8.83807653e-31 6.77184202e-23 8.28621098e-17
  1.58730838e-16 2.16361727e-07 4.00892750e-05 1.22095662e-04
  2.98335806e-07 3.32473775e-07 1.07885702e-04 7.28856833e-08
  1.01301691e-06 2.15638281e-07 7.36002227e-16 6.37870357e-07
  4.35240916e-04 5.63281162e-07 5.96986199e-03 3.25337942e-06
  2.59821718e-06 3.03435876e-09 6.06474350e-06 9.85560119e-01
  7.50468811e-03 3.36126573e-

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [188]:
np.argmax(prediction)

59

So the first letter predicted after the seed sequence is:

In [189]:
indices_char[np.argmax(prediction)]

's'

In [190]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(600):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [191]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

[' and work on anything you want as long a->s its not poetengetien  and the tey wery very bad theres. They just wanted to',
 'sat a week or theres to the people who hold a bit calked the cig pictures and it dasser oreals to we think about their',
 'create orssibilin. And I shought what we hind or energy from that puite pavions.they are aasoatly. Thats a lot of goades',
 'in tome people. And some of those people created structures to get the word out to ever more people. And lo that tian',
 'that we gave a pecief markenser. The things about interactions. Bnd the only reason these are more like for the part.',
 'Bvt in the madk if seis that weer te ara more']

In [192]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

[' and work on anything you want as long as its not part of your regular job. Work on anything you want. Engineers use',
 'this time to come up with a cool patch for code come up with an elegant hack. Then they present all of the stuff that',
 'theyve developed to their teammates to the rest of the company in this wild and woolly allhands meeting at the end of',
 'the day. Being Australians everybody has a beer. They call them FedEx Days. Why? Because you have to deliver something',
 'overnight. Its pretty not bad. Its a huge trademark violation but its pretty clever.  That one day of intense autonomy',
 'has produced a whole array of software fixes that might never have existed. Its worked so well that Atlassian has taken',
 'it to the next level with 20 time  done famously at Google  where engineers can spend 20 of their time working on',
 'anything they want. They have autonomy over their time their task their team their technique. Radical amounts of',
 'autonomy. And at Google as m